In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from scipy.stats.mstats import winsorize
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import ADASYN, SMOTE



df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mia_proyect/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mia_proyect/test.csv")

In [3]:
#Cambio de nombre de las columnas
df = df.rename(columns={
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal"
})
df_test = df_test.rename(columns={
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal"
})


In [4]:
#Remplazamos los -9 por nan

cols_num = df.select_dtypes(include=['int64', 'float64']).columns
for col in cols_num:
 df[cols_num] = df[cols_num].replace(-9, np.nan)

cols_num = df_test.select_dtypes(include=['int64', 'float64']).columns
for col in cols_num:
 df_test[cols_num] = df_test[cols_num].replace(-9, np.nan)



In [5]:
import pandas as pd
import numpy as np

def impute_missing_values(df_input):
    df = df_input.copy()

    numeric_cols_to_impute = [
        'tension_en_descanso', 'colesterol', 'azucar', 'latidos_por_minuto',
        'dolor_pecho_con_ejercicio', 'cambio_linea_corazon_ejercicio',
        'forma_linea_corazon_ejercicio', 'num_venas_grandes', 'estado_corazon_thal'
    ]

    for col in numeric_cols_to_impute:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Now fill missing values using median or mode
    df['tension_en_descanso'] = df['tension_en_descanso'].fillna(df['tension_en_descanso'].median())
    df['colesterol'] = df['colesterol'].fillna(df['colesterol'].median())
    df['azucar'] = df['azucar'].fillna(df['azucar'].mode()[0])
    df['latidos_por_minuto'] = df['latidos_por_minuto'].fillna(df['latidos_por_minuto'].median())
    df['dolor_pecho_con_ejercicio'] = df['dolor_pecho_con_ejercicio'].fillna(df['dolor_pecho_con_ejercicio'].mode()[0])
    df['cambio_linea_corazon_ejercicio'] = df['cambio_linea_corazon_ejercicio'].fillna(df['cambio_linea_corazon_ejercicio'].median())
    df['forma_linea_corazon_ejercicio'] = df['forma_linea_corazon_ejercicio'].fillna(df['forma_linea_corazon_ejercicio'].mode()[0])
    df['num_venas_grandes'] = df['num_venas_grandes'].fillna(df['num_venas_grandes'].mode()[0])
    df['estado_corazon_thal'] = df['estado_corazon_thal'].fillna(df['estado_corazon_thal'].mode()[0])

    return df


df_imputed = impute_missing_values(df.copy())
df_test_imputed = impute_missing_values(df_test.copy())

print("DataFrame (df) after imputation (head):")
display(df_imputed.head())
print("DataFrame (df_test) after imputation (head):")
display(df_test_imputed.head())

DataFrame (df) after imputation (head):


,edad,sexo,tipo_dolor_pecho,tension_en_descanso,colesterol,azucar,electro_en_descanso,latidos_por_minuto,dolor_pecho_con_ejercicio,cambio_linea_corazon_ejercicio,forma_linea_corazon_ejercicio,num_venas_grandes,estado_corazon_thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,-9.0,-9.0,7.0,2
2,63.0,1.0,4.0,140.0,0.0,0.0,2.0,149.0,0.0,2.0,1.0,-9.0,-9.0,2
3,52.0,0.0,2.0,140.0,-9.0,0.0,0.0,140.0,0.0,0.0,-9.0,-9.0,-9.0,0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3


DataFrame (df_test) after imputation (head):


,edad,sexo,tipo_dolor_pecho,tension_en_descanso,colesterol,azucar,electro_en_descanso,latidos_por_minuto,dolor_pecho_con_ejercicio,cambio_linea_corazon_ejercicio,forma_linea_corazon_ejercicio,num_venas_grandes,estado_corazon_thal
0,57.0,1.0,4.0,156.0,173.0,0.0,2,119.0,1.0,3.0,3.0,-9.0,-9.0
1,52.0,1.0,2.0,160.0,196.0,0.0,0.0,165.0,0.0,0.0,-9.0,-9.0,-9.0
2,48.0,1.0,2.0,100.0,-9.0,0.0,0.0,100.0,0.0,0.0,-9.0,-9.0,-9.0
3,62.0,1.0,4.0,115.0,0.0,0.0,0,128.0,1.0,2.5,3.0,-9.0,-9.0
4,51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import warnings


warnings.filterwarnings("ignore")  # evitar warnings por penalizaciones

X = df_imputed.drop('label', axis=1) # Drop the target column to get features
y = df_imputed['label'] # Select the target column

# Now, split the features and target into training and testing sets for model validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lista de configuraciones a probar
configuraciones = [
    {"penalty": "l2", "solver": "lbfgs"},
    {"penalty": "l1", "solver": "liblinear"},
    {"penalty": "elasticnet", "solver": "saga", "l1_ratio": 0.5},
    {"penalty": "none", "solver": "lbfgs"}
]

for config in configuraciones:
    print(f"\n🔍 Configuración: {config}")
    try:
        modelo = LogisticRegression(max_iter=1000, multi_class='auto', random_state=42, **config)
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        y_proba = modelo.predict_proba(X_test)

        print(classification_report(y_test, y_pred))
        print("F1 weighted:", f1_score(y_test, y_pred, average='weighted'))
        print("AUC:", roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted'))
    except Exception as e:
        print("❌ Error en esta configuración:", e)



🔍 Configuración: {'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.72      0.86      0.79        66
           1       0.41      0.42      0.41        31
           2       0.17      0.06      0.08        18
           3       0.28      0.36      0.31        22
           4       0.00      0.00      0.00        10

    accuracy                           0.54       147
   macro avg       0.31      0.34      0.32       147
weighted avg       0.47      0.54      0.50       147

F1 weighted: 0.4971786853767233
AUC: 0.784875048599252

🔍 Configuración: {'penalty': 'l1', 'solver': 'liblinear'}
              precision    recall  f1-score   support

           0       0.69      0.89      0.78        66
           1       0.38      0.35      0.37        31
           2       0.43      0.17      0.24        18
           3       0.29      0.32      0.30        22
           4       0.00      0.00      0.00        10

    accuracy    

In [7]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

modelos = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}

resultados = []

for nombre, modelo in modelos.items():
    try:
        modelo.fit(X_train, y_train)
        pred = modelo.predict(X_test)

        mae = mean_absolute_error(y_test, pred)
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        r2 = r2_score(y_test, pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        resultados.append({
            "modelo": nombre,
            "MAE": mae,
            "RMSE": rmse,
            "R2": r2,
            "F1": f1
        })

        print(f"\n✅ {nombre}")
        print(f"MAE:  {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"R²:   {r2:.3f}")
        print(f"F1:   {f1:.3f}")

    except Exception as e:
        print(f"\n❌ {nombre} falló:", e)

# Mostrar ranking ordenado
df_resultados = pd.DataFrame(resultados).sort_values("RMSE")
display(df_resultados)



✅ LinearRegression
MAE:  0.80
RMSE: 1.04
R²:   0.385
F1:   0.451

✅ Ridge
MAE:  0.80
RMSE: 1.04
R²:   0.385
F1:   0.451

✅ Lasso
MAE:  0.82
RMSE: 1.05
R²:   0.364
F1:   0.451

✅ DecisionTree
MAE:  0.96
RMSE: 1.43
R²:   -0.162
F1:   0.451

✅ RandomForest
MAE:  0.79
RMSE: 1.08
R²:   0.338
F1:   0.451

✅ XGBoost
MAE:  0.86
RMSE: 1.14
R²:   0.254
F1:   0.451


,modelo,MAE,RMSE,R2,F1
1,Ridge,0.799555,1.037691,0.385047,0.451017
0,LinearRegression,0.799661,1.037787,0.384933,0.451017
2,Lasso,0.823669,1.054998,0.364362,0.451017
4,RandomForest,0.792041,1.076719,0.337918,0.451017
5,XGBoost,0.861994,1.142866,0.254072,0.451017
3,DecisionTree,0.959184,1.426188,-0.161610,0.451017


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Modelos y sus grids
modelos_y_parametros = {
    "LinearRegression": (
        make_pipeline(StandardScaler(), LinearRegression()),
        {}
    ),
    "Ridge": (
        make_pipeline(StandardScaler(), Ridge()),
        {"ridge__alpha": [0.1, 1.0, 10.0]}
    ),
    "Lasso": (
        make_pipeline(StandardScaler(), Lasso(max_iter=5000)),
        {"lasso__alpha": [0.001, 0.01, 0.1, 1.0]}
    ),
    "DecisionTree": (
        DecisionTreeRegressor(random_state=42),
        {"max_depth": [3, 5, 7, None], "min_samples_split": [2, 5, 10]}
    ),
    "RandomForest": (
        RandomForestRegressor(random_state=42),
        {"n_estimators": [100, 200], "max_depth": [None, 5, 10]}
    ),
    "XGBoost": (
        XGBRegressor(random_state=42, verbosity=0),
        {"n_estimators": [100, 200], "max_depth": [3, 5, 7], "learning_rate": [0.01, 0.1]}
    )
}

resultados = []

for nombre, (modelo, grid) in modelos_y_parametros.items():
    print(f"\n🔍 Buscando mejores parámetros para {nombre}...")

    grid_search = GridSearchCV(modelo, grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    r2 = r2_score(y_test, pred)
    f1 = f1_score(y_test, y_pred, average='weighted')


    resultados.append({
        "modelo": nombre,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "mejores_parametros": grid_search.best_params_,
        "F1": f1

    })

# Mostrar resultados ordenados
df_resultados = pd.DataFrame(resultados).sort_values("RMSE")
pd.set_option("display.max_colwidth", None)
display(df_resultados)



🔍 Buscando mejores parámetros para LinearRegression...

🔍 Buscando mejores parámetros para Ridge...

🔍 Buscando mejores parámetros para Lasso...

🔍 Buscando mejores parámetros para DecisionTree...

🔍 Buscando mejores parámetros para RandomForest...

🔍 Buscando mejores parámetros para XGBoost...


,modelo,MAE,RMSE,R2,mejores_parametros,F1
4,RandomForest,0.776545,1.029926,0.394214,"{'max_depth': 5, 'n_estimators': 100}",0.451017
2,Lasso,0.799669,1.037700,0.385035,{'lasso__alpha': 0.001},0.451017
0,LinearRegression,0.799661,1.037787,0.384933,{},0.451017
1,Ridge,0.799982,1.038397,0.384209,{'ridge__alpha': 10.0},0.451017
5,XGBoost,0.785647,1.060038,0.358274,"{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}",0.451017
3,DecisionTree,0.810469,1.112986,0.292567,"{'max_depth': 5, 'min_samples_split': 10}",0.451017


In [9]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score, roc_auc_score

# Modelos base
modelo1 = LogisticRegression(max_iter=1000, class_weight='balanced')
modelo2 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
modelo3 = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Ensamblado con votación
ensamble = VotingClassifier(
    estimators=[
        ('lr', modelo1),
        ('rf', modelo2),
        ('xgb', modelo3)
    ],
    voting='soft'  # usa proba en lugar de votación dura
)

ensamble.fit(X_train, y_train)
y_pred = ensamble.predict(X_test)
y_proba = ensamble.predict_proba(X_test)

# Evaluación
print("🔍 Ensamblado VotingClassifier")
print(classification_report(y_test, y_pred))
print("F1 (weighted):", f1_score(y_test, y_pred, average='weighted'))
print("AUC (ovr):", roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted'))


🔍 Ensamblado VotingClassifier
              precision    recall  f1-score   support

           0       0.72      0.83      0.77        66
           1       0.41      0.29      0.34        31
           2       0.29      0.28      0.29        18
           3       0.23      0.32      0.27        22
           4       0.00      0.00      0.00        10

    accuracy                           0.52       147
   macro avg       0.33      0.34      0.33       147
weighted avg       0.48      0.52      0.49       147

F1 (weighted): 0.49470065657215506
AUC (ovr): 0.7851102331071863
